In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys

import nest_asyncio


sys.path.insert(0, os.path.abspath('..'))
nest_asyncio.apply()

In [2]:
import logging


logging.basicConfig(
    level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s'
)

### Prepare input file

In [3]:
from decouple import config
from huggingface_hub import AsyncInferenceClient


HUGGINGFACE_TOKEN = config('HUGGINGFACE_TOKEN')

MODEL_HUB_ID = 'microsoft/Phi-3-mini-4k-instruct'

client = AsyncInferenceClient(
    model=MODEL_HUB_ID,
    provider='hf-inference',
    timeout=None,
    api_key=HUGGINGFACE_TOKEN,
)

In [3]:
from math_rag.application.base.assistants import BaseAssistantInput, BaseAssistantOutput


class SomeInput(BaseAssistantInput):
    pass


class SomeOutput(BaseAssistantOutput):
    result: int

In [4]:
import json

from pathlib import Path

from math_rag.application.models.inference import (
    LLMBatchRequest,
    LLMBatchResult,
    LLMConversation,
    LLMMessage,
    LLMParams,
    LLMRequest,
)
from math_rag.infrastructure.mappings.inference.huggingface import (
    LLMRequestMapping,
    LLMResponseListMapping,
)


MODEL_HUB_ID = 'microsoft/Phi-3-mini-4k-instruct'
some_input = SomeInput()

# request = LLMRequest(
#     conversation=LLMConversation(
#         messages=[
#             LLMMessage(role='system', content='You are a helpful assistant.'),
#             LLMMessage(role='user', content='what is 2+2'),
#         ]
#     ),
#     params=LLMParams(
#         model=MODEL_HUB_ID,
#         temperature=0,
#         response_type=SomeOutput,
#         max_completion_tokens=10,
#         metadata={'input_id': str(some_input.id)},
#     ),
# )

# batch_request: LLMBatchRequest = LLMBatchRequest(requests=[request])

_requests = [
    LLMRequest(
        conversation=LLMConversation(
            messages=[
                LLMMessage(role='system', content='You are a helpful assistant.'),
                LLMMessage(role='user', content=f'what is {i}+2'),
            ]
        ),
        params=LLMParams(
            model=MODEL_HUB_ID,
            temperature=0,
            response_type=SomeOutput,
            max_completion_tokens=10,
            metadata={'input_id': str(some_input.id)},
        ),
    )
    for i in range(200)
]

batch_request: LLMBatchRequest = LLMBatchRequest(requests=_requests)

In [5]:
requests = [LLMRequestMapping.to_target(request) for request in batch_request.requests]
lines = [json.dumps(request, separators=(',', ':')) for request in requests]
jsonl_str = '\n'.join(lines)
jsonl_bytes = jsonl_str.encode('utf-8')

In [7]:
request_dict = json.loads(lines[0])

In [8]:
result = await client.chat_completion(**request_dict)
result

ChatCompletionOutput(choices=[ChatCompletionOutputComplete(finish_reason='stop', index=0, message=ChatCompletionOutputMessage(role='assistant', content='{ "result": 4 }', tool_call_id=None, tool_calls=None), logprobs=None)], created=1744125157, id='', model='microsoft/Phi-3-mini-4k-instruct', system_fingerprint='3.2.1-native', usage=ChatCompletionOutputUsage(completion_tokens=9, prompt_tokens=17, total_tokens=26), object='chat.completion')

In [40]:
response_list = LLMResponseListMapping.to_source(
    result,
    request_id=request.id,
    input_id=request_dict['extra_body']['input_id'],
    response_type=SomeOutput,
)
response_list

LLMResponseList(id=UUID('f7fd3b29-2772-468d-831f-b160bf1dfaa0'), request_id=UUID('cb02d5a3-bed8-4109-b06c-04d585081ef3'), responses=[LLMResponse(id=UUID('3976268b-adbf-494a-b88b-fda24781d477'), content=BoundAssistantOutput(id=UUID('bd7fdedd-f32a-4d7f-845d-0a21413828b6'), input_id=UUID('c882baa3-2047-4f65-b8df-bd176bba3b2c'), result=4), logprobs=None)])

In [ ]:
input_file_path = Path(f'.tmp/input_{batch_request.id}.jsonl')

with open(input_file_path, 'w') as input_file:
    for line in lines:
        input_file.write(line + '\n')

### Apptainer

In [5]:
from math_rag.infrastructure.containers import InfrastructureContainer


infrastructure_container = InfrastructureContainer()
infrastructure_container.init_resources()

tgi_batch_llm = infrastructure_container.tgi_batch_llm()

In [ ]:
await tgi_batch_llm.init_resources()

In [ ]:
res = await tgi_batch_llm.batch_generate(
    batch_request=batch_request,
    response_type=SomeOutput,
    poll_interval=3 * 60,
    max_tokens_per_day=None,
    max_num_retries=0,
)
res

2025-05-06 17:47:01,578 - INFO - Host canonicalization disabled
2025-05-06 17:47:01,578 - INFO - Opening SSH connection to login-gpu.hpc.srce.hr, port 22
2025-05-06 17:47:01,589 - INFO - [conn=19] Connected to SSH server at login-gpu.hpc.srce.hr, port 22
2025-05-06 17:47:01,589 - INFO - [conn=19]   Local address: 172.18.0.7, port 46056
2025-05-06 17:47:01,589 - INFO - [conn=19]   Peer address: 161.53.2.37, port 22
2025-05-06 17:47:01,634 - INFO - [conn=19] Beginning auth for user lpanic
2025-05-06 17:47:01,899 - INFO - [conn=19] Auth for user lpanic succeeded
2025-05-06 17:47:01,902 - INFO - [conn=19, chan=0] Requesting new SSH session
2025-05-06 17:47:01,930 - INFO - [conn=19, chan=0]   Subsystem: sftp
2025-05-06 17:47:01,938 - INFO - [conn=19, chan=0] Starting SFTP client
2025-05-06 17:47:02,143 - INFO - [conn=19, chan=0] Received exit status 0
2025-05-06 17:47:02,145 - INFO - [conn=19, chan=0] Received channel close
2025-05-06 17:47:02,147 - INFO - [conn=19, chan=0] SFTP client exit

LLMBatchResult(id=UUID('9df3488e-1a57-49d5-b324-b3f55333241e'), batch_request_id=UUID('a300045b-cc64-464b-afae-46524fab91e7'), response_lists=[LLMResponseList(id=UUID('d607ec85-3191-4033-8666-b4f06ee0bd15'), request_id=UUID('4a68f6e6-b00d-4b0a-ac8a-e6eb99d8290e'), responses=[LLMResponse(id=UUID('138ce650-b713-42e1-8642-a2219a2496f6'), content=BoundAssistantOutput(id=UUID('700bda8b-80fa-482b-aac3-56283f27db4c'), input_id=UUID('2108ee92-fc10-4671-9bce-b07818da5f6c'), result=37), logprobs=None)]), LLMResponseList(id=UUID('ffdf6555-81dd-44da-9c69-91202991fc6f'), request_id=UUID('4cba64d2-d5fc-4958-b965-959777e9b607'), responses=[LLMResponse(id=UUID('210db408-7c6a-4e2a-b0c5-00ecb0867973'), content=BoundAssistantOutput(id=UUID('0a01f70b-7f68-4a75-b289-5227387e36b0'), input_id=UUID('2108ee92-fc10-4671-9bce-b07818da5f6c'), result=56), logprobs=None)]), LLMResponseList(id=UUID('eb434c32-f4d1-4953-9f53-e7c9e6559c2c'), request_id=UUID('114b84e4-ab4c-4998-809f-a67bd238d3e1'), responses=[LLMRespons

In [9]:
from pathlib import Path


sftp_client = infrastructure_container.sftp_client()

await sftp_client.download(
    Path('tgi_default_root/error.log'), Path('../.tmp/error.log')
)

2025-05-05 09:25:03,554 - INFO - Host canonicalization disabled
2025-05-05 09:25:03,555 - INFO - Opening SSH connection to login-gpu.hpc.srce.hr, port 22
2025-05-05 09:25:03,574 - INFO - [conn=35] Connected to SSH server at login-gpu.hpc.srce.hr, port 22
2025-05-05 09:25:03,575 - INFO - [conn=35]   Local address: 172.18.0.7, port 55530
2025-05-05 09:25:03,575 - INFO - [conn=35]   Peer address: 161.53.2.37, port 22
2025-05-05 09:25:03,613 - INFO - [conn=35] Beginning auth for user lpanic
2025-05-05 09:25:03,879 - INFO - [conn=35] Auth for user lpanic succeeded
2025-05-05 09:25:03,880 - INFO - [conn=35, chan=0] Requesting new SSH session
2025-05-05 09:25:03,903 - INFO - [conn=35, chan=0]   Subsystem: sftp
2025-05-05 09:25:03,911 - INFO - [conn=35, chan=0] Starting SFTP client
2025-05-05 09:25:04,135 - INFO - [conn=35, chan=0] Received exit status 0
2025-05-05 09:25:04,136 - INFO - [conn=35, chan=0] Received channel close
2025-05-05 09:25:04,136 - INFO - [conn=35, chan=0] SFTP client exit

In [8]:
from pathlib import Path


sftp_client = infrastructure_container.sftp_client()

# tar -czvf snapshot.tar.gz data

# TODO download dir
await sftp_client.download(
    Path('tgi_default_root/snapshot.tar.gz'), Path('../.tmp/snapshot.tar.gz')
)

# tar -xzvf snapshot.tar.gz

2025-05-06 14:11:46,868 - INFO - Host canonicalization disabled
2025-05-06 14:11:46,868 - INFO - Opening SSH connection to login-gpu.hpc.srce.hr, port 22
2025-05-06 14:11:46,885 - INFO - [conn=39] Connected to SSH server at login-gpu.hpc.srce.hr, port 22
2025-05-06 14:11:46,885 - INFO - [conn=39]   Local address: 172.18.0.7, port 36530
2025-05-06 14:11:46,885 - INFO - [conn=39]   Peer address: 161.53.2.37, port 22
2025-05-06 14:11:46,918 - INFO - [conn=39] Beginning auth for user lpanic
2025-05-06 14:11:47,191 - INFO - [conn=39] Auth for user lpanic succeeded
2025-05-06 14:11:47,192 - INFO - [conn=39, chan=0] Requesting new SSH session
2025-05-06 14:11:47,217 - INFO - [conn=39, chan=0]   Subsystem: sftp
2025-05-06 14:11:47,225 - INFO - [conn=39, chan=0] Starting SFTP client
2025-05-06 14:11:47,470 - INFO - [conn=39, chan=0] Received exit status 0
2025-05-06 14:11:47,471 - INFO - [conn=39, chan=0] Received channel close
2025-05-06 14:11:47,471 - INFO - [conn=39, chan=0] SFTP client exit

In [ ]:
from os import environ

from decouple import config


environ['PBS_O_WORKDIR'] = '../.tmp'
environ['TGI_API_KEY'] = config('HUGGINGFACE_TOKEN')
environ['MODEL_HUB_ID'] = 'microsoft/Phi-3-mini-4k-instruct'

%run ../assets/hpc/hf/tgi/tgi_client.py